# Plot alternatives for Batch

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import pandas as pd
import numpy as np

import holoviews as hv
import hvplot.pandas

In [ ]:
import cellpy

In [ ]:
hv.extension("bokeh")
pd.set_option("display.max_columns", 500)

In [ ]:
# only for my MacBook
filename = "../testdata/hdf5/20160805_test001_45_cc.h5"
assert os.path.isfile(filename)
my_cell = cellpy.get(filename)

## Plotting summaries

In [ ]:
summary = my_cell.dataset.summary

In [ ]:
x = "Cycle_Index"
y = ["Charge_Capacity(mAh/g)", "Discharge_Capacity(mAh/g)"]

(summary.hvplot(x, y, kind="scatter") * summary.hvplot(x, y, kind="line"))

## Plotting raw data

In [ ]:
raw = my_cell.dataset.raw

In [ ]:
from holoviews import opts
from holoviews.plotting.links import RangeToolLink


def _rawplot(raw_curve, title="Voltage versus time"):
    tgt = raw_curve.relabel(title).opts(
        width=800,
        height=300,
        labelled=["y"],
        # tools=["pan","box_zoom", "reset"],
        active_tools=["pan"],
    )
    src = raw_curve.opts(width=800, height=100, yaxis=None, default_tools=[])

    RangeToolLink(src, tgt)

    layout = (tgt + src).cols(1)
    layout.opts(opts.Layout(shared_axes=False, merge_tools=False))
    return layout

In [ ]:
def rawplot(raw, y=("Voltage", "Voltage (V vs Li/Li+)"), title="Voltage versus time"):
    raw["Test_Time_Hrs"] = raw["Test_Time"] / 3600
    x = ("Test_Time_Hrs", "Time (hours)")
    raw_curve = hv.Curve(
        raw,
        x,
        y,
    )
    layout = _rawplot(raw_curve, title=title)
    return layout

In [ ]:
rawplot(raw, y=("Current", "current (A)"), title="current")

In [ ]:
title = "Voltage versus time"

raw["Test_Time_Hrs"] = raw["Test_Time"] / 3600
x = ("Test_Time_Hrs", "Time (hours)")
y1 = ("Voltage", "Voltage (V vs Li/Li+)")
y2 = ("Current", "Current (A)")
raw_curve1 = hv.Curve(raw, x, y1)
raw_curve2 = hv.Curve(raw, x, y2)

tgt1 = raw_curve1.relabel(title).opts(
    width=800,
    height=300,
    labelled=["y"],
    # tools=["pan","box_zoom", "reset"],
    active_tools=["pan"],
)

tgt2 = raw_curve2.relabel("Current vs time").opts(
    width=800,
    height=300,
    labelled=["y"],
)

src = raw_curve1.opts(width=800, height=100, yaxis=None, default_tools=[])

RangeToolLink(src, tgt1)

# funker ikke (?) å linke to figurer til samme "rangetoolpanel"
linked = tgt1 + src
linked.opts(opts.Layout(shared_axes=False, merge_tools=False))
layout = (tgt2 + linked).cols(1)
layout.opts(opts.Layout(shared_axes=True, merge_tools=False))

In [ ]:
x = ("Test_Time_Hrs", "Time (hours)")
layout_ex = [hv.Curve(raw, x, y) for y in ["Voltage", "Current"]]

In [ ]:
vo1 = hv.Curve(raw, x, "Voltage", group="one", label="1")
vo2 = hv.Curve(raw, x, "Voltage", group="one", label="2")
vo3 = hv.Curve(raw, x, "Current", group="two", label="3")
d1 = {
    "v1": vo1,
    "v2": vo2,
    "c1": vo3,
}

In [ ]:
nn = hv.NdLayout(d1)
nn.opts(opts.NdLayout(tabs=True, width=500))

In [ ]:
ocv_up = my_cell.get_ocv(direction="up")
ocv_up.sample(5)

ocv_up = ocv_up[ocv_up.Cycle_Index < 6]

In [ ]:
ocv_ovrl = (
    hv.Curve(
        ocv_up,
        kdims=["Step_Time", "Cycle_Index"],
        vdims=["Voltage"],
        label="cen59_01",
        group="background",
    )
    .groupby("Cycle_Index")
    .overlay()
)

In [ ]:
ocv_hmap = hv.Curve(
    ocv_up, kdims=["Step_Time"], vdims=["Voltage", "Cycle_Index"], label="cen59_01"
).groupby("Cycle_Index")

In [ ]:
hv.Layout(ocv_ovrl * ocv_hmap).opts(
    opts.NdOverlay(show_legend=False),
    opts.Curve(show_legend=False, width=600),
    opts.Curve("background", alpha=0.2),
    opts.Layout(show_title=False, toolbar="right"),
)

In [ ]:
%%opts Layout [tabs=True] Curve [width=800]
newlayout = hv.Layout(vo1+vo2+vo3, )
newlayout